# 2. CoLLiE 的使用分析

> 2.1 &ensp; 案例：指令微调 MOSS
> 
> 2.2 &ensp; CoLLiE 的 Config 模块
> 
> 2.3 &ensp; CoLLiE 的 Dataset 和 Model
> 
> 2.4 &ensp; CoLLiE 的 Evaluator 和 Metric
> 
> 2.5 &ensp; CoLLiE 的 Trainer 模块

## 2.1 &ensp; 案例：指令微调 MOSS

&ensp; &ensp; 在上一节中，我们已经介绍了 CoLLiE 提出的背景、实现的功能 以及 包含的模块。通过上一节的叙述，想必大家已经可以配置好环境、成功地安装 CoLLiE，并且了解了 CoLLiE 的大致结构，遇到一些的功能需求可以找到对应的代码文件，尝试自行了解相关功能是如何实现的了。

&ensp; &ensp; 在这一节中，我们将介绍 **CoLLiE 的使用流程** 以及 **基础模块的使用**，包括 **Config**、**Dataset**、**Evaluator**、**Trainer**。而正所谓实践出真知，在详细介绍上述模块的使用细节与基础原理前，本小节将先通过一个 CoLLiE 的使用案例，告诉大家 CoLLiE 是如何使用、相关代码是如何运行起来的；接着在后续小节中，详细介绍各个部分代码的运行原理、对应模块的使用细节。

&ensp; &ensp; 本教程使用的CoLLiE使用案例是，使用 LOMO 算法（优化算法），零冗余优化器 ZeRO-3（并行策略），全参数微调 MOSS-003（基座模型），使其过拟合一句话（微调任务）：输入 'Collie is a python package for'，输出 'finetuning large language models .'（训练数据）。完整代码文件见 CoLLiE 代码仓库中的 [examples/finetune_moss_for_training.py](https://github.com/OpenLMLab/collie/blob/dev/examples/finetune_moss_for_training.py)，切换到所在路径后，在命令行通过如下命令即可完成运行。

```sh
CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --standalone --nproc_per_node=4 finetune_moss_for_training.py
```

&ensp; &ensp; 这里重点解释一下这行命令的含义。首先，**CUDA_VISIBLE_DEVICES=0,1,2,3**，表示使用当前服务器上的0至3号显卡。接着，**torchrun xxx.py**，表示以分布式的形式启动 `finetune_moss_for_training.py`，即在0至3号显卡上并行运行 `finetune_moss_for_training.py`；至于torchrun，只要python环境中安装pytorch包，即可以运行。剩下的 **--standalone** 和 **--nproc_per_node=4** 都是 torchrun 命令的参数：前者表示在1台服务器上运行；后者表示当前服务器（node）拥有的显卡数量（每个显卡对应一个进程 process）。

&ensp; &ensp; 关于pytorch分布式相关的内容，这里还需要额外补充 **world_size** 和 **rank** 两个概念。world_size 表示 整个分布式程序对应的子进程数量，rank 表示 每个子进程的标识号；对于上述命令对应的程序，由于其在1台服务器的4张显卡上运行，1卡对应1个子进程，所以world_size=4，0至3号显卡上的子进程rank分别是0到3。至于如何**结束这条程序**，可以在找到子进程id号之后，**关闭任意一个子进程即可**；关闭子进程，父进程会回收剩余子进程并结束；若关闭父进程，则每个子进程仍需要手动关闭。

&ensp; &ensp; 关于pytorch分布式相关的更多内容，可以参考 [pytorch官方文档](https://pytorch.org/docs/stable/index.html) 中 [关于torchrun的介绍](https://pytorch.org/docs/stable/elastic/run.html)。在接下来的教程中，我们将打开 `finetune_moss_for_training.py`，一点点讲解前文所述的代码模块、并行策略、优化算法是什么，以及 CoLLiE 是如何实现它们的。

## 2.2 &ensp; CoLLiE 的 Config 模块

&ensp; &ensp; 打开 `finetune_moss_for_training.py`，除了最开始的import部分，第一个用到的CoLLiE模块就是CollieConfig。**CoLLie 的配置模块 CollieConfig 是整个 CoLLiE 的核心**，CoLLie 的高度集成化归功于对 CollieConfig 配置文件的集中统一管理。**CoLLiE 几乎所有的组件都受到 CollieConfig 的控制**，包括模型架构（CollieConfig.model_config）、并行策略（CollieConfig.dp_size, CollieConfig.pp_size, CollieConfig.tp_size）、微调方法（CollieConfig.peft_config）等；CollieConfig 涉及的全部配置参数 以及 对应的功能描述 如下表所示。

| 名称 | 描述 |
|:----|:----|
| seed | 随机数种子，整数 默认 42 | 
| dp_size | 数据并行粒度，整数 默认 1，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html#3.1-%E2%80%82-数据并行（Data-Parallel）) | 
| pp_size | 流水线并行粒度，整数 默认 1，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html#3.2-%E2%80%82-流水线并行（Pipeline-Parallel）) | 
| tp_size | 张量并行粒度，整数 默认 1，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html#3.3-%E2%80%82-张量并行（Tensor-Parallel）) | 
| pp_partition_method | 流水线并行切分方式，默认 'parameters'，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html#3.2-%E2%80%82-流水线并行（Pipeline-Parallel）). | 
| train_epochs | 训练 epoch 数量，整数 默认 100 | 
| eval_per_n_steps | 多少 step 一次评测，整数 默认 0 | 
| eval_per_n_epochs | 多少 epoch 一次评测，整数 默认 0 | 
| train_micro_batch_size | 训练 batch 大小，整数 默认 1（流水线并行：作为 micro_batch 大小，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html#3.2-%E2%80%82-流水线并行（Pipeline-Parallel）)） | 
| gradient_accumulation_steps | 多少 backward 一次 step，整数 默认 1（流水线并行：决定 train_batch 大小，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html#3.2-%E2%80%82-流水线并行（Pipeline-Parallel）)） | 
| eval_batch_size | 测试 batch 大小，整数 默认 1 | 
| checkpointing | 是否使用 activation checkpointing，默认 True |
| use_flash | 是否使用 flash attention 进行自注意力加速，默认 True | 
| dropout | dropout 大小，浮点数 默认 0.0 | 
| init_method | 参数初始化方法，可选值 'none'（默认）、'normal'、'kaiming_normal'、'kaiming_uniform'等 | 
| low_cpu_mem_usage | 是否在模型初始化阶段尝试减少 CPU 占用，默认 True | 
| ds_config | 指定 deepspeed 参数，字典型 或 json文件名，涉及零冗余优化器，详见 [tutorial-3](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-3-parallel.html#3.5-%E2%80%82-零冗余优化器（ZeRO）) | 
| model_config | 指定模型架构相关的配置项，默认 [transformers.PretrainedConfig()](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/configuration#transformers.PretrainedConfig) | 
| peft_config | 指定模型参数高效微调方法，默认 [peft.PeftConfig()](https://huggingface.co/docs/peft/main/en/package_reference/config#peft.PeftConfig) | 
| quantization_config | 指定模型量化方法，默认 [transformers.BitsAndBytesConfig()](https://huggingface.co/docs/transformers/main_classes/quantization#transformers.BitsAndBytesConfig) | 

&ensp; &ensp; 由于在 CollieConfig 的所有的配置项中，模型架构相关的配置是较为繁琐但又相对固定的，因此 CollieConfig 提供了与 transformers.PretrainedConfig 类似的 **from_pretrained 方法初始化 CollieConfig**，并对 CollieConfig.model_config 赋值。例如，在 `finetune_moss_for_training.py` 中，首先明确模型架构为"fnlp/moss-moon-003-sft"，接着使用 CollieConfig.from_pretrained 快速初始化，再定义并行相关参数（需要注意的是，由于是4卡运行，虽然三个并行粒度全部设为1，但数据并行粒度dp_size会自动设为4，详见 [tutorial-3](collie-tutorial-3-parallel.html#3.4-%E2%80%82-整合：3D并行（3D-Parallel）)），最后定义训练相关参数，如 训练轮数、评测频率、batch大小。

``` python
from collie.config import CollieConfig

pretrained_model = "fnlp/moss-moon-003-sft"  

config = CollieConfig.from_pretrained(pretrained_model, trust_remote_code=True)
config.dp_size = 1  # 4
config.tp_size = 1
config.pp_size = 1
config.train_epochs = 1
config.eval_per_n_steps = 0
config.eval_per_n_epochs = 1 
config.train_micro_batch_size = 2
config.eval_batch_size = 1
```

## 2.3 &ensp; CoLLiE 的 Dataset 和 Model

&ensp; &ensp; 在 `finetune_moss_for_training.py` 中，在通过 CollieConfig 制定配置参数之后，接下来就是依次加载 **数据集 Dataset**、初始化 **模型 Model**；二者也是本小节将要详细介绍的 CoLLiE 模块。

&ensp; &ensp; 首先介绍的是 **CoLLiE 自定义的模型基类 CollieModelForCausalLM**。我们知道 CoLLiE 最关键的功能是同时实现了对多种并行策略的支持，但是 张量并行 和 流水线并行 对模型架构的严格要求，普通的 torch.nn.Modul 或 transformers.PreTrainedModel 无法实现上述并行策略；因此，在 CoLLiE 中，我们自定义了支持多种并行方案的模型基类 CollieModelForCausalLM。**使用 CoLLiE 训练或评测模型**，**必须使用  CollieModelForCausalLM 或其子类 初始化模型**；如果强行使用 torch.nn.Modul 或 transformers.PreTrainedModel 训练，仅支持 数据并行 和 零冗余优化器。

&ensp; &ensp; CollieModelForCausalLM 继承并重写了一些较为流行的模型，例如 LLaMA、MOSS、ChatGLM/ChatGLM2、InternLM，方便用户直接调用，如下表所示；

| 名称 | 描述 |
|:----|:----|
| CollieModelForCausalLM | CoLLiE 自定义模型基类，兼容 DP PP TP ZeRO，支持 训练、测试、生成 | 
| ChatGLMForCausalLM | CoLLiE 自定义 ChatGLM，支持 | 
| ChatGLM2ForCausalLM | CoLLiE 自定义 ChatGLM2，支持 | 
| InternLMForCausalLM | CoLLiE 自定义 InternLM，支持 | 
| LlamaForCausalLM | CoLLiE 自定义 LLaMA，包括 7B、13B、30B、65B 版本 | 
| MossForCausalLM | CoLLiE 自定义 MOSS，支持 | 
| Moss003MoonForCausalLM | CoLLiE 自定义 MOSS-moon，支持 Moss-moon-003 | 

重写的过程中，我们保证接口尽可能和 transformers 一致，如下表所示，方便熟悉 transformers 的用户能快速上手使用。

| 名称 | 描述 |
|:----|:----|
| from_config | | 
| from_pretrained | | 
| forward | | 
| generate | | 
| main_input_name | | 
| training | | 

<!-- 问张硕，你觉得还有哪些功能需要额外添加的 -->

例如，在 `finetune_moss_for_training.py` 脚本中，在指定模型类别并初始化 CollieConfig 后，即可以通过 **CollieModelForCausalLM.from_pretrained**，即这里的 Moss003MoonForCausalLM.from_pretrained，初始化 Moss-moon-003 模型，并从 huggingface 下载预训练参数并完成加载，加载过程遵循传入的 CollieConfig。

``` python
from collie.models.moss_moon import Moss003MoonForCausalLM

# pretrained_model = "fnlp/moss-moon-003-sft"  
# config = CollieConfig.from_pretrained(pretrained_model, trust_remote_code=True)

model = Moss003MoonForCausalLM.from_pretrained(pretrained_model, config=config)
``` 

&ensp; &ensp; 接下来介绍的是 **CoLLiE 自定义的数据集 CollieDataset**，**CollieDataset 实现了自动的数据格式转换**。在 CoLLiE 的使用过程中，可以使用 [datasets.load_dataset](https://huggingface.co/docs/datasets/v2.13.1/en/package_reference/loading_methods#datasets.load_dataset)，不一定涉及到 CollieDataset；但是无论如何，都必须要实现输入数据格式的转换，即 **将每笔数据转化为字典**，并且 **该字典的键值 需要与 模型CollieModel 和 评测指标Metric的输入 一致**（至于如何保持一致、为什么要保持一致，会在后文有详细的叙述）；而这就是 CollieDataset 实现的功能。CoLLiE 共定义了三种 CollieDataset，CollieDatasetForTraining/CollieDatasetForPerplexity、CollieDatasetForGeneration、CollieDatasetForClassfication，**三种 CollieDataset 的功能描述与输入输出格式** 如下表所示（注：表格中，xx 表示某个元素，[xx] 表示某个元素列表）。

| 名称 | 描述 |
|:----|:----|
| CollieDatasetForTraining | 所有CollieDataset的基类，行为与CollieDatasetForPerplexity完全相同 |
| CollieDatasetForPerplexity | 用于训练或测试LLM输出困惑度的CollieDataset，输入两种形式，输出一种形式 <br /> 输入：字典列表，字典格式：{'text': xx} 或 {'input_ids': xx, 'output': xx} <br /> 输出：字典列表，字典格式：{'input_ids': xx, 'attention_mask': xx, 'labels': xx, } |
| CollieDatasetForGeneration | 用于测试LLM执行生成任务效果的CollieDataset，输入一种形式，输出一种形式 <br /> 输入：字典列表，字典格式：{'text': xx, 'target': xx} <br /> 输出：字典列表，字典格式：{'input_ids': xx, 'attention_mask': xx, 'labels': xx, 'target': xx, } |
| CollieDatasetForClassfication | 用于测试LLM执行分类任务效果的CollieDataset，输入一种形式，输出两种形式 <br /> 输入：字典列表，字典格式：{'input': xx, 'output': [xx], 'labels': xx}：输出：字典列表，分两种情况 <br /> &emsp; harness style：字典格式：{'input_ids': [xx], 'attention_mask': [xx], 'labels': [xx], 'target': xx, } <br /> &emsp; helm style：字典格式：{'input_ids': xx, 'attention_mask': xx, 'output': [xx], 'labels': xx, 'target': xx, } |

&ensp; &ensp; 上表简要描述了三种 CollieDataset 的输入输出格式，接下来我们将介绍 **三种 CollieDataset 的输入输出对应关系**，整体上如下图所示。对于 CollieDatasetForPerplexity，


&ensp; &ensp; 对于 CollieDatasetForGeneration，

<div style="position:relative;width:100%;" align="center">
  <img src="images/collie-datasets.png" width="68%" align="center" />
</div>

&ensp; &ensp; 对于 CollieDatasetForClassfication，

&ensp; &ensp; 在 `finetune_moss_for_training.py` 脚本中，

``` python
from collie.data import CollieDatasetForTraining
from collie.data import CollieDataLoader

train_dataset = [
    {
        'input': 'Collie is a python package for ',
        'output': 'finetuning large language models.'
    } for _ in range(10000)
]
train_dataset = CollieDatasetForTraining(train_dataset, tokenizer)
eval_dataset = train_dataset[:32]
``` 

## 2.4 &ensp; CoLLiE 的 Evaluator 和 Metric

&ensp; &ensp; 在 `finetune_moss_for_training.py` 中，在调用 CollieConfig 制定配置参数、调用 CollieModelForCausalLM 初始化模型、调用 CollieDatasetForTraining 加载数据集之后，接下来就是 初始化 **监控器 Monitor**、**评测指标 Metric**、**评测模块 Evaluator**、**训练模块 Trainer**，并进行训练和测试。其中，监控器 Monitor 较为次要，将在最后的 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-lomo-peft.html#4.4-%E2%80%82-CoLLiE-的-Monitor-模块) 详细介绍；优化器 Optim、训练模块 Trainer 则在稍后的 [tutorial-2](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-2-lomo-peft.html#2.5-%E2%80%82-CoLLiE-的-训练模块) 详细展开。剩下的 评测指标 Metric、评测模块 Evaluator，我们会在本小节中给出其详细的参数列表与功能意义；此外，**CoLLiE 的 数据集 Dataset、评测指标 Metric、评测模块 Evaluator 存在紧密的对应关系**，我们将在详细介绍其参数功能后，重点分析之。

&ensp; &ensp; evaluator模块列表

| 名称 | 描述 |
|:----|:----|
| Evaluator | | 
| EvaluatorForPerplexity | | 
| EvaluatorForClassfication | | 
| EvaluatorForGeneration | | 

&ensp; &ensp; metric模块列表

| 名称 | 描述 |
|:----|:----|
| BaseMetric | |
| DecodeMetric | |
| AccuracyMetric | |
| PPLMetric | |
| BleuMetric | |
| RougeMetric | |
| ClassifyFPreRecMetric | |

&ensp; &ensp; 如前文所述，**CoLLiE 的 数据集 Dataset、评测指标 Metric、评测模块 Evaluator 存在紧密的对应关系**。

&ensp; &ensp; CollieEvaluatorForPerplexity

<div style="position:relative;width:100%;" align="center">
  <img src="images/collie-perplexity.png" width="68%" align="center" />
</div>

&ensp; &ensp; CollieEvaluatorForGeneration

<div style="position:relative;width:100%;" align="center">
  <img src="images/collie-generation.png" width="68%" align="center" />
</div>

&ensp; &ensp; CollieEvaluatorForClassification

<div style="position:relative;width:100%;" align="center">
  <img src="images/collie-classification.png" width="68%" align="center" />
</div>

&ensp; &ensp; 在 `finetune_moss_for_training.py` 脚本中，

``` python
from collie.controller.evaluator import EvaluatorForPerplexity, EvaluatorForGeneration

from collie.metrics import PPLMetric, DecodeMetric

evaluator_ppl = EvaluatorForPerplexity(
    model=model, config=config, dataset=eval_dataset,
    monitors=[EvalMonitor(config), ], metrics={'ppl': PPLMetric(), }
)

evaluator_decode = EvaluatorForGeneration(
    model=model, config=config, tokenizer=tokenizer, dataset=eval_dataset,
    monitors=[EvalMonitor(config), ], metrics={'decode': DecodeMetric(), }
)
``` 

&ensp; &ensp; 其他一些交代内容

## 2.5 &ensp; CoLLiE 的 Trainer 模块

&ensp; &ensp; optim简单介绍

&ensp; &ensp; trainer参数列表

| 名称 | 描述 |
|:----|:----|
| model | 待训练模型，要求是 ColliModelForCausalLM 类型 | 
| config | 训练配置参数，即初始化之后的 CollieConfig | 
| tokenizer | 模型对应分词器，要求是 PreTrainedTokenizerBase 类型 | 
| loss_fn | 损失函数，默认 GPTLMLoss()，即 封装后的 nn.CrossEntropyLoss() | 
| train_fn | 训练函数，默认 None，允许传入自定义函数控制训练过程 | 
| eval_fn | 评测函数，默认 None，允许传入自定义函数控制评测过程 | 
| optimizer | 优化器，默认 None，CoLLiE 实现 LOMO 优化器 详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-lomo-peft.html#4.1-%E2%80%82-CoLLiE-实现-LOMO-算法) | 
| lr_scheduler | 优化器控制，默认 None，参考 [torch.optim.lr_scheduler](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate) | 
| train_dataset | 训练数据集，torch.utils.data.Dataset 或 CollieDatasetForTraining 类型 | 
| eval_dataset | 训练数据集，torch.utils.data.Dataset 或 CollieDatasetForTraining 类型 | 
| callbacks | 回调函数列表，自定义模型训练过程，详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-lomo-peft.html#4.3-%E2%80%82-CoLLiE-的-Callback-模块) | 
| train_dataset_collate_fn | 负责训练数据padding的函数，默认 ColliePadder() | 
| eval_dataset_collate_fn | 负责训练测试padding的函数，默认 ColliePadder(padding_left=True) | 
| data_provider | 数据提供器，默认 None，BaseProvider 类型，详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-lomo-peft.html#4.5-%E2%80%82-CoLLiE-的-Server-模块) | 
| monitors | 监控器列表，默认 None，BaseMonitor 类型，详见 [tutorial-4](https://openlmlab-collie.readthedocs.io/zh_CN/latest/tutorials/collie-tutorial-4-lomo-peft.html#4.4-%E2%80%82-CoLLiE-的-Monitor-模块) | 
| metrics | 评测指标字典，例如 {'acc': AccuracyMetric()} | 
| evaluators | 评测模块列表，例如 [EvaluatorForClassfication(), ] | 

&ensp; &ensp; 在 `finetune_moss_for_training.py` 脚本中，

``` python
from collie.controller.trainer import Trainer

trainer = Trainer(
    model=model,config=config,train_dataset=train_dataset,
    loss_fn=GPTLMLoss(-100),optimizer=optimizer,
    monitors=monitors, evaluators=[evaluator_ppl, evaluator_decode]
)

trainer.train()
```